## Alphavantage for Stock Fundamentals

In [1]:
import requests
import json
import pandas as pd
import numpy as np
from datetime import date, timedelta
import time
import os
from dotenv import load_dotenv; load_dotenv()

alpha_key = os.getenv('ALPHA_KEY1')
base_url = 'https://www.alphavantage.co/query/'
start_date = date(2018,1,1)
end_date = date.today()
tickers = ["TSLA", "BABA", "HD", "NVDA", "JNJ", "JPM", "NKE", "TWTR", "AAPL", "AMZN", "XOM", "AMC", "PLUG", "PG", "PFE"]

In [2]:
# returns date iterator
def datetime_range(start=None, end=date.today()):
  delta = end - start
  for i in range(delta.days + 1):
    yield start + timedelta(days=i)

In [3]:
# Function for getting historical prices
def prices_df(ticker, start_date=date.today() - timedelta(days=365), end_date=date.today()):
  params = {'function': 'TIME_SERIES_DAILY', 'symbol': ticker, 'outputsize': 'full', 'apikey': alpha_key}
  resp = requests.get(base_url, params=params)
  data = resp.json()
  data = data['Time Series (Daily)']

  columns=['OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOLUME', 'ticker', 'date']
  df = pd.DataFrame(columns=columns, index=pd.to_datetime([]))
  # print(data['2021-02-02'])
  for date in list(datetime_range(start=start_date, end=end_date)):
    date = date.strftime('%Y-%m-%d')
    try:
      values = list(data[date].values())
      values.extend([ticker, date])
      df.loc[date] = pd.Series([float(value) if value != ticker and value != date else value for value in values], columns)
    except KeyError:
      continue
  df['date']= pd.to_datetime(df['date'])
  df.set_index(['ticker', 'date'], inplace=True)
  return df


# Price Signals


In [5]:
# Price signals helper function
def price_signals(df_prices):
    """
    Calculate price-signals for a single stock.
    Use sf.apply() with this function for multiple stocks.
    
    :param df_prices:
        Pandas DataFrame with raw share-prices for a SINGLE stock.
    
    :return:
        Pandas DataFrame with price-signals.
    """
    
    # Create new DataFrame for the signals.
    # Setting the index improves performance.
    df_signals = pd.DataFrame(index=df_prices.index)

    # df_signals['ticker'] = df_prices['ticker']

    # Moving Average for past 20 days.
    df_signals['MAVG_20'] = df_prices['CLOSE'].rolling(window=20).mean()

    # Moving Average for past 200 days.
    df_signals['MAVG_200'] = df_prices['CLOSE'].rolling(window=200).mean()

    # Exponential Moving Average for past 20 days.
    df_signals['EMA'] = df_prices['CLOSE'].ewm(span=20).mean()
    
    # Moving Average Convergence Divergence for 12 and 26 days.
    # https://en.wikipedia.org/wiki/MACD
    df_signals['MACD'] = df_prices['CLOSE'].ewm(span=12).mean() - df_prices['CLOSE'].ewm(span=26).mean()
    
    # MACD with extra smoothing by Exp. Moving Average for 9 days.
    df_signals['MACD_EMA'] = df_signals['MACD'].ewm(span=9).mean()

    # The last trading volume relative to 20-day moving average.
    df_signals['REL_VOL'] = np.log(df_prices['VOLUME'] / df_prices['VOLUME'].rolling(window=20).mean())
    
    return df_signals

In [6]:
df_prices = prices_df(ticker='JNJ', start_date=start_date)
df_prices

OPEN    HIGH     LOW   CLOSE     VOLUME
ticker date                                                 
JNJ    2018-01-02  139.66  139.95  138.72  139.23  6835739.0
       2018-01-03  139.36  140.68  138.90  140.56  5241755.0
       2018-01-04  140.45  141.14  140.22  140.55  4777518.0
       2018-01-05  140.69  141.82  140.28  141.71  5900718.0
       2018-01-08  141.70  142.00  140.92  141.89  4980217.0
...                   ...     ...     ...     ...        ...
       2021-12-06  160.19  163.52  159.83  162.94  9692041.0
       2021-12-07  164.00  164.27  162.41  163.36  8311254.0
       2021-12-08  163.47  164.57  162.64  164.34  6216391.0
       2021-12-09  164.18  165.98  163.49  165.90  8574177.0
       2021-12-10  165.57  166.69  164.65  165.49  6324820.0

[994 rows x 5 columns]

In [7]:
df_signals = price_signals(df_prices=df_prices)
df_signals.head()

MAVG_20  MAVG_200         EMA      MACD  MACD_EMA  REL_VOL
ticker date                                                                  
JNJ    2018-01-02      NaN       NaN  139.230000  0.000000  0.000000      NaN
       2018-01-03      NaN       NaN  139.928250  0.029840  0.016578      NaN
       2018-01-04      NaN       NaN  140.156553  0.037712  0.025239      NaN
       2018-01-05      NaN       NaN  140.605009  0.082249  0.044551      NaN
       2018-01-08      NaN       NaN  140.915837  0.111881  0.064581      NaN

In [8]:
final_prices = pd.DataFrame(index=pd.to_datetime([]))
for ticker in tickers:
  print(ticker)
  df_prices = prices_df(ticker, start_date, end_date)
  final_prices = final_prices.append(df_prices)
  time.sleep(9.5)

TSLA
BABA
HD
NVDA
JNJ
JPM
NKE
TWTR
AAPL
AMZN
XOM
AMC
PLUG
PG
PFE


In [ ]:
# final_prices.set_index(['ticker'], inplace=True )
final_prices.head()

OPEN    HIGH     LOW   CLOSE     VOLUME
ticker date                                                 
TSLA   2018-01-02  312.00  322.11  311.00  320.53  4326988.0
       2018-01-03  321.00  325.25  315.55  317.25  4185665.0
       2018-01-04  312.87  318.55  305.68  314.62  9915932.0
       2018-01-05  316.62  317.24  312.00  316.58  4578729.0
       2018-01-08  316.00  337.02  315.50  336.41  9777059.0

In [ ]:
final_signals = pd.DataFrame(index=pd.to_datetime([]))
for ticker, df_grp in final_prices.groupby('ticker'):
  df_signals = price_signals(df_prices=df_grp)
  final_signals = final_signals.append(df_signals)

Exporting stock price dataframe and price signals dataframe to CSV

In [ ]:
final_prices.to_csv("data/quantitative/stock_price.csv")
final_signals.to_csv("data/quantitative/price_signals.csv")

## Resampling with asfreq




In [ ]:
def _asfreq(df_grp):
    # Remove TICKER from the MultiIndex.
    df_grp = df_grp.reset_index('ticker', drop=True)
    
    # Perform the operation on this group.
    df_result = df_grp.asfreq(freq='D', method='ffill')

    return df_result

# Split the DataFrame into sub-groups and apply the _asfreq()
# function on each of those sub-groups, and then glue the
# results back together into a single DataFrame. e.g.
# df_income.groupby(TICKER).apply(_asfreq)

# Financial Signals

In [ ]:
# Get quarterly financials of a company (Income, Balance Sheets and cashflow)
def df_income(ticker, start_date=date.today() - timedelta(days=365), end_date=date.today()):
  params = {'function': 'INCOME_STATEMENT', 'symbol': ticker, 'apikey': alpha_key}
  resp = requests.get(base_url, params=params)
  data = resp.json()
  data = data['quarterlyReports']

  # print(data)
  time_period_financials = list()
  for date in list(datetime_range(start=start_date, end=end_date)):
      date = date.strftime('%Y-%m-%d')
      found_value = [income_deets for income_deets in data if income_deets['fiscalDateEnding'] == date]
      if len(found_value) > 0:
        time_period_financials.append(found_value[0])
  
  # Creating dataframe to return
  columns = list(time_period_financials[0].keys())
  df = pd.DataFrame(time_period_financials , columns=columns)
  df['ticker'] = ticker
  df['dateEnding']= pd.to_datetime(df['fiscalDateEnding'])
  df.drop(columns=['fiscalDateEnding', 'reportedCurrency'], inplace=True)
  df.set_index(['ticker', 'dateEnding'], inplace=True)

  # Ensuring all numbers are converted from string to numeric
  for column in df.columns:
    column_vals = pd.to_numeric(df[column], errors='ignore')
    df[column] = column_vals
    
  return df

# Get quarterly balance sheet details for a ticker
def df_balance_sheet(ticker, start_date=date.today() - timedelta(days=365), end_date=date.today()):
  params = {'function': 'BALANCE_SHEET', 'symbol': ticker, 'apikey': alpha_key}
  resp = requests.get(base_url, params=params)
  data = resp.json()
  data = data['quarterlyReports']

  # print(data)
  time_period_financials = list()
  for date in list(datetime_range(start=start_date, end=end_date)):
      date = date.strftime('%Y-%m-%d')
      found_value = [income_deets for income_deets in data if income_deets['fiscalDateEnding'] == date]
      if len(found_value) > 0:
        time_period_financials.append(found_value[0])

  columns = list(time_period_financials[0].keys())
  df = pd.DataFrame(time_period_financials , columns=columns)
  df['ticker'] = ticker
  df['dateEnding']= pd.to_datetime(df['fiscalDateEnding'])
  df.drop(columns=['fiscalDateEnding', 'reportedCurrency'], inplace=True)
  df.set_index(['ticker', 'dateEnding'], inplace=True)

  # Ensuring all numbers are converted from string to numeric
  for column in df.columns:
    column_vals = pd.to_numeric(df[column], errors='ignore')
    df[column] = column_vals

  return df

# Get quarterly cashflow details for a ticker
def df_cashflow(ticker, start_date=date.today() - timedelta(days=365), end_date=date.today()):
  params = {'function': 'CASH_FLOW', 'symbol': ticker, 'apikey': alpha_key}
  resp = requests.get(base_url, params=params)
  data = resp.json()
  data = data['quarterlyReports']

  # print(data)
  time_period_financials = list()
  for date in list(datetime_range(start=start_date, end=end_date)):
      date = date.strftime('%Y-%m-%d')
      found_value = [income_deets for income_deets in data if income_deets['fiscalDateEnding'] == date]
      if len(found_value) > 0:
        time_period_financials.append(found_value[0])

  columns = list(time_period_financials[0].keys())
  df = pd.DataFrame(time_period_financials , columns=columns)
  df['ticker'] = ticker
  df['dateEnding']= pd.to_datetime(df['fiscalDateEnding'])
  df.drop(columns=['fiscalDateEnding', 'reportedCurrency'], inplace=True)
  # pd.to_numeric(df,)
  df.set_index(['ticker', 'dateEnding'], inplace=True)
  
  # Ensuring all numbers are converted from string to numeric
  for column in df.columns:
    column_vals = pd.to_numeric(df[column], errors='ignore')
    df[column] = column_vals
  
  return df

In [ ]:
apple_cashflow_df = df_cashflow('AAPL')
apple_cashflow_df

operatingCashflow  ...    netIncome
ticker dateEnding                     ...             
AAPL   2020-12-31        38763000000  ...  28755000000
       2021-03-31        23981000000  ...  23630000000
       2021-06-30        21094000000  ...  21744000000
       2021-09-30        20200000000  ...  20551000000

[4 rows x 27 columns]

In [ ]:
# multiticker income, balance_sheet and cashflow statements
income_df = pd.DataFrame(index=pd.to_datetime([]))
balance_df = pd.DataFrame(index=pd.to_datetime([]))
cashflow_df = pd.DataFrame(index=pd.to_datetime([]))

for ticker in tickers:
  print(ticker)
  # Quarterly Income
  alpha_key = os.getenv('ALPHA_KEY1')
  ticker_income = df_income(ticker, start_date, end_date)
  # Quarterly Balance Sheet
  alpha_key = os.getenv('ALPHA_KEY2')
  ticker_balance_sheet = df_balance_sheet(ticker, start_date, end_date)
  # Quarterly Cashflow
  alpha_key = os.getenv('ALPHA_KEY3')
  ticker_cashflow = df_cashflow(ticker, start_date, end_date)

  # adding to final dataframes
  income_df = income_df.append(ticker_income)
  balance_df = balance_df.append(ticker_balance_sheet)
  cashflow_df = cashflow_df.append(ticker_cashflow)
  time.sleep(11)

TSLA
BABA
HD
NVDA
JNJ
JPM
NKE
TWTR
AAPL
AMZN
XOM
AMC
PLUG
PG
PFE


In [ ]:
# Resampling using asfreq
income_df2 = income_df.groupby('ticker').apply(_asfreq)
balance_df2 = balance_df.groupby('ticker').apply(_asfreq)
cashflow_df2 = cashflow_df.groupby('ticker').apply(_asfreq)

In [ ]:
# Exporting to csv files
income_df2.to_csv("data/quantitative/income_statements.csv")
balance_df2.to_csv("data/quantitative/balance_sheets.csv")
cashflow_df2.to_csv("data/quantitative/cashflows.csv")

## Creating financials signals using company financial data

In [ ]:
def fin_signals(income_df, balance_df):
    """
    Calculate financial signals for a single stock.
    
    :param df:
        Pandas DataFrame with required data from
        Income Statements, Balance Sheets, etc.
        Assumed to be TTM-data.
    
    :return:
        Pandas DataFrame with financial signals.
    """
    
    # Create new DataFrame for the signals.
    # Setting the index improves performance.
    df_signals = pd.DataFrame(index=income_df.index)

    # Net Profit Margin.
    df_signals['NET_PROFIT_MARGIN'] = income_df['netIncome'] / income_df['totalRevenue']
    
    # Return on Assets.
    df_signals['ROA'] = income_df['netIncome'] / balance_df['totalAssets'].shift(4)
    
    # Return on Equity.
    df_signals['ROE'] = income_df['netIncome'] / balance_df['totalShareholderEquity'].shift(4)

    return df_signals

In [ ]:
fin_signals(income_df.loc['AAPL'], balance_df.loc['AAPL'])

,NET_PROFIT_MARGIN,ROA,ROE
dateEnding,,,
2018-03-31,0.226082,NaN,NaN
2018-06-30,0.216258,NaN,NaN
2018-09-30,0.224563,NaN,NaN
2018-12-31,0.239331,NaN,NaN
2019-03-31,0.202807,0.031458,0.091119
2019-06-30,0.189713,0.028763,0.0873779
2019-09-30,0.213710,0.037422,0.127731
2019-12-31,0.244260,0.059499,0.188613
2020-03-31,0.195444,0.032892,0.106263


In [ ]:
def coerce_num(df):
  df = df.copy()
  for column in df.columns:
      column_vals = pd.to_numeric(df[column], errors='coerce')
      df[column] = column_vals
  return df

income_df3 = coerce_num(income_df)
balance_df3 = coerce_num(balance_df)
cashflow_df3 = coerce_num(cashflow_df)

In [ ]:
all_fin_signals = pd.DataFrame(index=pd.to_datetime([]))
for ticker, df_grp in income_df3.groupby('ticker'):
  print(ticker)
  df_fin_signal = fin_signals(income_df=df_grp, balance_df=balance_df3.loc[ticker])
  all_fin_signals = all_fin_signals.append(df_fin_signal)

AAPL
AMC
AMZN
BABA
HD
JNJ
JPM
NKE
NVDA
PFE
PG
PLUG
TSLA
TWTR
XOM


In [ ]:
final_fin_signals = all_fin_signals.groupby('ticker').apply(_asfreq)
final_fin_signals

NET_PROFIT_MARGIN       ROA       ROE
ticker dateEnding                                       
AAPL   2018-03-31           0.226082       NaN       NaN
       2018-04-01           0.226082       NaN       NaN
       2018-04-02           0.226082       NaN       NaN
       2018-04-03           0.226082       NaN       NaN
       2018-04-04           0.226082       NaN       NaN
...                              ...       ...       ...
XOM    2021-09-26           0.069233  0.012974  0.026029
       2021-09-27           0.069233  0.012974  0.026029
       2021-09-28           0.069233  0.012974  0.026029
       2021-09-29           0.069233  0.012974  0.026029
       2021-09-30           0.091481  0.018852  0.038050

[19387 rows x 3 columns]

In [ ]:
# Convert to CSV
final_fin_signals.to_csv("data/quantitative/financial_signals.csv")

## Valuation Signals

In [ ]:
balance_df.columns

Index(['totalAssets', 'totalCurrentAssets',
       'cashAndCashEquivalentsAtCarryingValue', 'cashAndShortTermInvestments',
       'inventory', 'currentNetReceivables', 'totalNonCurrentAssets',
       'propertyPlantEquipment', 'accumulatedDepreciationAmortizationPPE',
       'intangibleAssets', 'intangibleAssetsExcludingGoodwill', 'goodwill',
       'investments', 'longTermInvestments', 'shortTermInvestments',
       'otherCurrentAssets', 'otherNonCurrrentAssets', 'totalLiabilities',
       'totalCurrentLiabilities', 'currentAccountsPayable', 'deferredRevenue',
       'currentDebt', 'shortTermDebt', 'totalNonCurrentLiabilities',
       'capitalLeaseObligations', 'longTermDebt', 'currentLongTermDebt',
       'longTermDebtNoncurrent', 'shortLongTermDebtTotal',
       'otherCurrentLiabilities', 'otherNonCurrentLiabilities',
       'totalShareholderEquity', 'treasuryStock', 'retainedEarnings',
       'commonStock', 'commonStockSharesOutstanding'],
      dtype='object')